In [417]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

from IPython.core.display import display, HTML

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time, os
import sys

import pickle
import random

import re

In [2]:
# urls for women categories
url_women = []

for page in range(2, 175):
    url = "https://www.fragrancex.com/products/_cid_perfume__category.html?currentPage=" + str(page) + "&searchSortExpression=0"
    url_women.append(url)

In [3]:
url_first_women = 'https://www.fragrancex.com/products/_cid_perfume__category.html'
url_women.insert(0, url_first_women)

In [4]:
# urls for men categories
url_men = []

for page in range(1, 101):
    url = "https://www.fragrancex.com/products/_cid_cologne__category.html?currentPage="+ str(page) + "&searchSortExpression=0"
    url_men.append(url)

In [5]:
url_first_men = 'https://www.fragrancex.com/products/_cid_cologne__category.html'
url_men.insert(0, url_first_men)

In [320]:
# Save into pickle for future analysis
with open('url_list_women.pickle', 'wb') as to_write:
    pickle.dump(url_women, to_write)

In [321]:
with open('url_list_men.pickle', 'wb') as to_write:
    pickle.dump(url_men, to_write)

In [6]:
chromedriver = '/Applications/chromedriver' # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [7]:
driver = webdriver.Chrome(chromedriver)

In [23]:
#Extract urls of each item- women category
user_agent = {'User-Agent': 'Mozilla/5.0'}

details_urls_women = []

for url in url_women:
    driver.get(url)
    time.sleep(5+np.random.rand())
    soup = BeautifulSoup(driver.page_source, "html.parser")
    try: 
        atag=soup.find_all('a', class_ =  'animate product-image__link')
        for e in atag:
            url_sub = e.get("href")
            url_sub = 'https://www.fragrancex.com' + url_sub
            details_urls_women.append(url_sub)
        
    except Exception as e:
        time.sleep(5+np.random.rand())
        pass
      
    
    
    
    

In [24]:
driver.close()

In [28]:
driver = webdriver.Chrome(chromedriver)

In [29]:
#Extract urls of each item- men category
user_agent = {'User-Agent': 'Mozilla/5.0'}

details_urls_men = []

for url in url_men:
    driver.get(url)
    time.sleep(5+np.random.rand())
    soup = BeautifulSoup(driver.page_source, "html.parser")
    try: 
        atag=soup.find_all('a', class_ =  'animate product-image__link')
        for e in atag:
            url_sub = e.get("href")
            url_sub = 'https://www.fragrancex.com' + url_sub
            details_urls_men.append(url_sub)
        
    except Exception as e:
        time.sleep(5+np.random.rand())
        pass
      

In [311]:
driver.close()

In [32]:
len(details_urls_men)

6026

In [446]:
with open('url_items_women.pickle', 'wb') as to_write:
    pickle.dump(details_urls_women, to_write)

In [323]:
with open('url_items_men.pickle', 'wb') as to_write:
    pickle.dump(details_urls_men, to_write)

In [309]:
driver = webdriver.Chrome(chromedriver)

In [348]:
#download html files -women's perfume
for url in details_urls_women:
    item_id = url.split("_")[3] + "_" + url.split("_")[4]
    driver.get(url)
    page_source = driver.page_source
    
    with open('html_women' + '/' + 'perfume_' + item_id + '.html', 'w') as f:
        f.write(page_source)
        time.sleep(1+np.random.rand())

In [284]:
driver.close()


In [343]:
driver = webdriver.Chrome(chromedriver)


In [344]:
#download html files - men's cologne
for url in details_urls_men[5967:len(details_urls_men)+1]:
    item_id = url.split("_")[3] + "_" + url.split("_")[4]
    driver.get(url)
    page_source = driver.page_source
    
    with open('html_men' + '/' + 'perfume_' + item_id + '.html', 'w') as f:
        f.write(page_source)
        time.sleep(1+np.random.rand())

In [345]:
driver.close()


In [346]:
#pull features from each html file
file_name_list_w = os.listdir('html_women')

In [418]:
product_ids = []
product_names = []
reviews = []
description_list = []
url_list = []
image_list = []
brand_list = []

In [419]:
for file_name in file_name_list_w:
    file_html = open('html_women' + '/' + file_name, 'r', encoding='utf-8')
    file_ = file_html.read()
    soup = BeautifulSoup(file_, "html.parser")
        
    if 'Access denied' in soup.find('title').text:
        pass
    else:
        product_id = file_name.split(".")[0]
        
        if soup.find_all('link', rel="canonical"):
            url = soup.find_all('link', rel="canonical")[0].get('href')
        else:
            url = 'Missing'
            
        if soup.find_all('meta', property='og:image'):
            image_url = soup.find_all('meta', property='og:image')[0].get('content')
        else:
            image_url = 'Missing'
            
        if soup.find_all("a", class_ = 'ga-product-brand product-header-link'):
            brand_name = soup.find_all("a", class_ = 'ga-product-brand product-header-link')[0].text
        else:
            brand_name = 'Missing'
        
        if soup.find_all('span', class_= 'product-name-short'):
            product_name = soup.find_all('span', class_= 'product-name-short')[0].text
        else:
            product_name = 'Missing'
            
        if soup.find_all('div', class_="teaser__points counter"):
            review = soup.find_all('div', class_="teaser__points counter")[0].text.strip()
        else:
            review = 'Missing'
        
        description = ''
            # Not all items have the 3 tags
        if soup.find_all('span', itemprop="Description"):
            first_text = soup.find_all('span', itemprop="Description")[0].text.strip()
        else:
            first_text = ''
        if soup.find_all('span', id="product-read-more-text-mq1"):
            mid_text = soup.find_all('span', id="product-read-more-text-mq1")[0].text
        else:
            mid_text = ''
        if soup.find_all('span', id="product-read-more-text"):
            last_text = soup.find_all('span', id="product-read-more-text")[0].text
        else:
            last_text = ''
        
        description = description + first_text + mid_text + last_text
        description = description.strip()
        
        
        product_ids.append(product_id)
        product_names.append(product_name)
        reviews.append(review)
        description_list.append(description)
        url_list.append(url)
        image_list.append(image_url)
        brand_list.append(brand_name)


In [420]:
print(len(product_ids))
print(len(product_names))
print(len(reviews))
print(len(description_list))
print(len(url_list))
print(len(image_list))
print(len(brand_list))

8224
8224
8224
8224
8224
8224
8224


In [421]:
#Create a dict
df_keys = ['product_id', 'product_name', 'product_description','brand_name','review','url', 'image_url']   

df_dic = dict(zip(df_keys,[product_ids,
                           product_names,
                           description_list,
                           brand_list,
                           reviews,
                           url_list,
                           image_list
                           ]))

In [422]:
#Covert dict to dataframe
df_women = pd.DataFrame(df_dic)
df_women.head()

,product_id,product_name,product_description,brand_name,review,url,image_url
0,perfume_l-am-pid_80223w,La Nuit Tresor Nude Perfume,"La Nuit Tresor Nude Perfume by Lancome, Feel i...",Lancome,0.0,https://www.fragrancex.com/products/_cid_perfu...,https://img.fragrancex.com/images/products/par...
1,perfume_m-am-pid_68830w,Miss Varens Perfume,"Miss Varens Perfume by Ulric De Varens, Launch...",Ulric De Varens,3.7,https://www.fragrancex.com/products/_cid_perfu...,https://img.fragrancex.com/images/products/par...
2,perfume_h-am-pid_1478w,Hodo Perfume,"Hodo Perfume by Fragluxe, Captivating and myst...",Fragluxe,5.0,https://www.fragrancex.com/products/_cid_perfu...,https://img.fragrancex.com/images/products/par...
3,perfume_p-am-pid_77114w,Personal Selection Two Perfume,Personal Selection Two Perfume by Fred Hayman ...,Fred Hayman,0.0,https://www.fragrancex.com/products/_cid_perfu...,Missing
4,perfume_b-am-pid_70988w,Bronze Goddess Perfume,"Bronze Goddess Perfume by Estee Lauder, Bronze...",Estee Lauder,4.6,https://www.fragrancex.com/products/_cid_perfu...,https://img.fragrancex.com/images/products/par...


In [423]:
with open('df_women.pickle', 'wb') as to_write:
    pickle.dump(df_women, to_write)

In [428]:
file_name_list_m = os.listdir('html_men')

In [429]:
product_ids = []
product_names = []
reviews = []
description_list = []
url_list = []
image_list = []
brand_list = []

In [430]:
for file_name in file_name_list_m:
    file_html = open('html_men' + '/' + file_name, 'r', encoding='utf-8')
    file_ = file_html.read()
    soup = BeautifulSoup(file_, "html.parser")
        
    if 'Access denied' in soup.find('title').text:
        pass
    else:
        product_id = file_name.split(".")[0]
        
        if soup.find_all('link', rel="canonical"):
            url = soup.find_all('link', rel="canonical")[0].get('href')
        else:
            url = 'Missing'
            
        if soup.find_all('meta', property='og:image'):
            image_url = soup.find_all('meta', property='og:image')[0].get('content')
        else:
            image_url = 'Missing'
            
        if soup.find_all("a", class_ = 'ga-product-brand product-header-link'):
            brand_name = soup.find_all("a", class_ = 'ga-product-brand product-header-link')[0].text
        else:
            brand_name = 'Missing'
        
        if soup.find_all('span', class_= 'product-name-short'):
            product_name = soup.find_all('span', class_= 'product-name-short')[0].text
        else:
            product_name = 'Missing'
            
        if soup.find_all('div', class_="teaser__points counter"):
            review = soup.find_all('div', class_="teaser__points counter")[0].text.strip()
        else:
            review = 'Missing'
        
        description = ''
            # Not all items have the 3 tags
        if soup.find_all('span', itemprop="Description"):
            first_text = soup.find_all('span', itemprop="Description")[0].text.strip()
        else:
            first_text = ''
        if soup.find_all('span', id="product-read-more-text-mq1"):
            mid_text = soup.find_all('span', id="product-read-more-text-mq1")[0].text
        else:
            mid_text = ''
        if soup.find_all('span', id="product-read-more-text"):
            last_text = soup.find_all('span', id="product-read-more-text")[0].text
        else:
            last_text = ''
        
        description = description + first_text + mid_text + last_text
        description = description.strip()
        
        
        product_ids.append(product_id)
        product_names.append(product_name)
        reviews.append(review)
        description_list.append(description)
        url_list.append(url)
        image_list.append(image_url)
        brand_list.append(brand_name)

In [431]:
print(len(product_ids))
print(len(product_names))
print(len(reviews))
print(len(description_list))
print(len(url_list))
print(len(image_list))
print(len(brand_list))

5963
5963
5963
5963
5963
5963
5963


In [432]:
df_keys = ['product_id', 'product_name', 'product_description','brand_name','review','url', 'image_url']   

df_dic = dict(zip(df_keys,[product_ids,
                           product_names,
                           description_list,
                           brand_list,
                           reviews,
                           url_list,
                           image_list
                           ]))

In [433]:
df_men = pd.DataFrame(df_dic)
df_men.head()

,product_id,product_name,product_description,brand_name,review,url,image_url
0,perfume_m-am-pid_78163m,Mercedes Benz The Move Cologne,Mercedes Benz The Move Cologne by Mercedes Ben...,Mercedes Benz,4.6,https://www.fragrancex.com/products/_cid_colog...,https://img.fragrancex.com/images/products/par...
1,perfume_j-am-pid_69458m,Jaguar Excellence Cologne,"Jaguar Excellence Cologne by Jaguar, Released ...",Jaguar,5.0,https://www.fragrancex.com/products/_cid_colog...,https://img.fragrancex.com/images/products/par...
2,perfume_s-am-pid_1225m,Stanley Blacker Cologne,Stanley Blacker Cologne by Stanley Blacker All...,Stanley Blacker,0.0,https://www.fragrancex.com/products/_cid_colog...,https://img.fragrancex.com/images/products/par...
3,perfume_1-am-pid_62325m,90210 Extreme Sport Cologne,"90210 Extreme Sport Cologne by Torand, Present...",Torand,0.0,https://www.fragrancex.com/products/_cid_colog...,Missing
4,perfume_b-am-pid_68722m,Bod Man 24k Cologne,"Bod Man 24K Cologne by Parfums De Coeur, Go fo...",Parfums De Coeur,4.9,https://www.fragrancex.com/products/_cid_colog...,https://img.fragrancex.com/images/products/par...


In [434]:
with open('df_men.pickle', 'wb') as to_write:
    pickle.dump(df_men, to_write)

In [435]:
df_men['department'] = "Men's Cologne"
df_men.head()

,product_id,product_name,product_description,brand_name,review,url,image_url,department
0,perfume_m-am-pid_78163m,Mercedes Benz The Move Cologne,Mercedes Benz The Move Cologne by Mercedes Ben...,Mercedes Benz,4.6,https://www.fragrancex.com/products/_cid_colog...,https://img.fragrancex.com/images/products/par...,Men's Cologne
1,perfume_j-am-pid_69458m,Jaguar Excellence Cologne,"Jaguar Excellence Cologne by Jaguar, Released ...",Jaguar,5.0,https://www.fragrancex.com/products/_cid_colog...,https://img.fragrancex.com/images/products/par...,Men's Cologne
2,perfume_s-am-pid_1225m,Stanley Blacker Cologne,Stanley Blacker Cologne by Stanley Blacker All...,Stanley Blacker,0.0,https://www.fragrancex.com/products/_cid_colog...,https://img.fragrancex.com/images/products/par...,Men's Cologne
3,perfume_1-am-pid_62325m,90210 Extreme Sport Cologne,"90210 Extreme Sport Cologne by Torand, Present...",Torand,0.0,https://www.fragrancex.com/products/_cid_colog...,Missing,Men's Cologne
4,perfume_b-am-pid_68722m,Bod Man 24k Cologne,"Bod Man 24K Cologne by Parfums De Coeur, Go fo...",Parfums De Coeur,4.9,https://www.fragrancex.com/products/_cid_colog...,https://img.fragrancex.com/images/products/par...,Men's Cologne


In [436]:
df_women['department'] = "Women's Perfume"
df_women.head()

,product_id,product_name,product_description,brand_name,review,url,image_url,department
0,perfume_l-am-pid_80223w,La Nuit Tresor Nude Perfume,"La Nuit Tresor Nude Perfume by Lancome, Feel i...",Lancome,0.0,https://www.fragrancex.com/products/_cid_perfu...,https://img.fragrancex.com/images/products/par...,Women's Perfume
1,perfume_m-am-pid_68830w,Miss Varens Perfume,"Miss Varens Perfume by Ulric De Varens, Launch...",Ulric De Varens,3.7,https://www.fragrancex.com/products/_cid_perfu...,https://img.fragrancex.com/images/products/par...,Women's Perfume
2,perfume_h-am-pid_1478w,Hodo Perfume,"Hodo Perfume by Fragluxe, Captivating and myst...",Fragluxe,5.0,https://www.fragrancex.com/products/_cid_perfu...,https://img.fragrancex.com/images/products/par...,Women's Perfume
3,perfume_p-am-pid_77114w,Personal Selection Two Perfume,Personal Selection Two Perfume by Fred Hayman ...,Fred Hayman,0.0,https://www.fragrancex.com/products/_cid_perfu...,Missing,Women's Perfume
4,perfume_b-am-pid_70988w,Bronze Goddess Perfume,"Bronze Goddess Perfume by Estee Lauder, Bronze...",Estee Lauder,4.6,https://www.fragrancex.com/products/_cid_perfu...,https://img.fragrancex.com/images/products/par...,Women's Perfume


In [439]:
df = pd.concat([df_women, df_men], ignore_index = True)

In [444]:
with open('df.pickle', 'wb') as to_write:
    pickle.dump(df, to_write)

product_id                                       perfume_m-am-pid_68830w
product_name                                         Miss Varens Perfume
product_description    Miss Varens Perfume by Ulric De Varens, Launch...
brand_name                                               Ulric De Varens
review                                                               3.7
url                    https://www.fragrancex.com/products/_cid_perfu...
image_url              https://img.fragrancex.com/images/products/par...
department                                               Women's Perfume
Name: 1, dtype: object